<a href="https://colab.research.google.com/github/DRose1991/Viscous-Shear-Cosmology-Simulation/blob/main/VSC_MCMC_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install emcee corner

import numpy as np
import emcee
import matplotlib.pyplot as plt
import corner

# ==========================================
# 1. REAL DATA: COSMIC CHRONOMETERS (Moresco et al.)
# ==========================================
# Redshift (z), Hubble Parameter H(z), Error
cc_data = np.array([
    [0.070, 69.0, 19.6], [0.090, 69.0, 12.0], [0.120, 68.6, 26.2],
    [0.170, 83.0, 8.0],  [0.179, 75.0, 4.0],  [0.199, 75.0, 5.0],
    [0.200, 72.9, 29.6], [0.270, 77.0, 14.0], [0.280, 88.8, 36.6],
    [0.350, 82.7, 8.4],  [0.352, 83.0, 14.0], [0.380, 81.5, 1.9],
    [0.400, 95.0, 17.0], [0.400, 82.0, 5.6],  [0.425, 87.1, 11.2],
    [0.440, 82.6, 7.8],  [0.470, 89.0, 50.0], [0.478, 80.9, 9.0],
    [0.480, 97.0, 62.0], [0.593, 104.0, 13.0], [0.680, 92.0, 8.0],
    [0.781, 105.0, 12.0],[0.875, 125.0, 17.0], [0.880, 90.0, 40.0],
    [0.900, 117.0, 23.0],[1.037, 154.0, 20.0], [1.300, 168.0, 17.0],
    [1.363, 160.0, 33.6],[1.430, 177.0, 18.0], [1.530, 140.0, 14.0],
    [1.750, 202.0, 40.0],[1.965, 186.5, 50.4]
])

z_obs = cc_data[:, 0]
Hz_obs = cc_data[:, 1]
Hz_err = cc_data[:, 2]

# ==========================================
# 2. THE PHYSICAL MODEL (Viscous Friedman)
# ==========================================
# VSC assumes effective pressure P_eff = P - 3*xi*H
# If xi = constant (simplest bulk viscosity), it acts like a decay term.
# For this audit, we fit: H(z) = H0 * sqrt( Omega_m(1+z)^3 + Omega_visc )
# where Omega_visc proxies the viscosity contribution.

def Hz_model(z, H0, Om, O_visc):
    # Enforce flat universe constraint if assuming Omega_k = 0?
    # For now, let O_visc float to see if it mimics Dark Energy (Lambda ~ 0.7)
    E_sq = Om * (1 + z)**3 + O_visc
    return H0 * np.sqrt(np.abs(E_sq)) # abs to prevent NaN during random walk

# ==========================================
# 3. STATISTICAL PIPELINE (MCMC)
# ==========================================

def log_likelihood(theta, z, y, yerr):
    H0, Om, O_visc = theta
    model = Hz_model(z, H0, Om, O_visc)
    sigma2 = yerr**2
    return -0.5 * np.sum((y - model)**2 / sigma2 + np.log(sigma2))

def log_prior(theta):
    H0, Om, O_visc = theta
    # FLAT PRIORS (No cheating)
    if 50.0 < H0 < 100.0 and 0.0 < Om < 1.0 and 0.0 < O_visc < 1.5:
        return 0.0
    return -np.inf

def log_probability(theta, z, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, z, y, yerr)

# Initialize Walkers
pos = [67.0, 0.3, 0.7] + 1e-4 * np.random.randn(32, 3)
nwalkers, ndim = pos.shape

print("Running MCMC on real data...")
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, args=(z_obs, Hz_obs, Hz_err))
sampler.run_mcmc(pos, 5000, progress=True)

# Analysis
flat_samples = sampler.get_chain(discard=100, thin=15, flat=True)
print("\n--- RESULTS ---")
labels = ["H0", "Omega_m", "Omega_visc"]
for i in range(ndim):
    mcmc = np.percentile(flat_samples[:, i], [16, 50, 84])
    q = np.diff(mcmc)
    print(f"{labels[i]}: {mcmc[1]:.3f} (+{q[1]:.3f} / -{q[0]:.3f})")

# corner.corner(flat_samples, labels=labels)
# plt.show()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 3.1 MB/s eta 0:00:00
Running MCMC on real data...


100%|██████████| 5000/5000 [00:07<00:00, 633.06it/s]


--- RESULTS ---
H0: 60.686 (+17.445 / -8.069)
Omega_m: 0.390 (+0.146 / -0.155)
Omega_visc: 0.780 (+0.306 / -0.306)
